# Humanloop RAG Evaluation Walkthrough
The goal of this notebook is to demonstrate how to take an existing RAG pipeline and integrate Humanloop in order to:
1. Setup logging for both your [Prompt](https://humanloop.com/docs/v5/concepts/prompts) and retriever [Tool](https://humanloop.com/docs/v5/concepts/prompts) so that you can easily track the versions of these components.
2. Create a [Dataset](https://humanloop.com/docs/v5/concepts/prompts) and run Evaluations to benchmark the performance of your RAG pipeline.
3. Configure [Evaluators](https://humanloop.com/docs/v5/concepts/evaluators) for monitoring your RAG pipeline in production.


## What is Humanloop?
Humanloop is an interactive development environment designed to streamline the entire lifecycle of LLM app development. It serves as a central hub where AI, Product, and Engineering teams can collaborate on Prompt management, Evaluation and Monitoring workflows. 


## What is RAG?
RAG stands for Retrieval Augmented Generation.
- **Retrieval** - Getting the relevant information from a larger data source for a given a query.
- **Augmented** - Using the retrieved information as input to an LLM.
- **Generation** - Generating an output from the model given the input.

In practise, it remains an effective way to exploit LLMs for things like question answering, summarization, and more, where the data source is too large to fit in the context window of the LLM, or where providing the full data source for each query is not cost-effective.


## What are the major challenges with RAG?
Implementing RAG and other similar flows complicates the process of [Prompt Engineering](https://humanloop.com/blog/prompt-engineering-101) because you expand the design space of your application. There are lots of choices you need to make around the retrieval and Prompt components that can significantly impact the performance of your overall application. For example,
- How do you select the data source?
- How should it be chunked up and indexed?
- What embedding and retrieval model should you use?
- How should you combine the retrieved information with the query?
- What should your system Prompt be? 
- Which model should you use?
- What should your system message be?
etc...

The process of versioning, evaluating and monitoring your pipeline therefore needs to consider both the retrieval and generation components. This is where Humanloop can help.


# Example RAG Pipeline

We first need a reference RAG implementation. Our use case will be Q&A over a corpus of medical documents.

- **Dataset**: we'll use a version of the [MedQA dataset](https://huggingface.co/datasets/bigbio/med_qa) from Hugging Face. This is a multiple choice question answering problem based on the United States Medical License Exams (USMLE), with reference text books that contain the required information to answer the questions.
- **Retriever**: we're going to use [Chroma](https://docs.trychroma.com/getting-started) as a simple local vector DB with their default embedding model `all-MiniLM-L6-v2`. You can replace this with your favorite retrieval system.
- **Prompt**: **the Prompt will be managed in code**, populated with the users question and the context retrieved from the Retriever and sent to [OpenAI](https://platform.openai.com/docs/api-reference/introduction) to generate the answer.

### Where to store your Prompts?

Generally speaking, when the engineering/applied AI teams are mainly responsible for managing the details of the Prompt, then the pattern of storing or constructing the Prompt in code works well. This is the pattern we follow in this tutorial. 

However, if the Product/Domain Expert teams are more involved in Prompt engineering and management, then the Prompt can instead be managed on Humanloop and retrieved or called by your code - this workflow lies outside the scope of this tutorial and we cover it separately. 

## Complete Prerequisites

### Install packages
We use poetry to manage dependencies:

In [ ]:
!poetry install

### Initialise the SDKs

You will need to set your OpenAI API key in the  `.env` file in the root of the repo. You can retrieve your API key from your [OpenAI account](https://platform.openai.com/api-keys).


In [1]:
# Set up dependencies
from dotenv import load_dotenv
import os
from chromadb import chromadb
from openai import OpenAI

import pandas as pd

# load .env file that contains API keys
load_dotenv()

# init clients
chroma = chromadb.Client()
openai = OpenAI(api_key=os.getenv("OPENAI_KEY"))


### Set up the Vector DB
This involves loading the data from the MedQA dataset and embedding the data within a collection in Chroma. This will take a couple of minutes to complete.

In [2]:
# init collection into which we will add documents
collection = chroma.get_or_create_collection(name="MedQA")

# load knowledge base
knowledge_base = pd.read_parquet("../../assets/sources/textbooks.parquet")
knowledge_base = knowledge_base.sample(5, random_state=42)


# Add to Chroma - will by default use local vector DB and model all-MiniLM-L6-v2
collection.add(
    documents=knowledge_base["contents"].to_list(),
    ids=knowledge_base["id"].to_list(),
)

### Define the Prompt
We define a simple prompt template that has variables for the question, answer options and retrieved data.

It is generally good practise to define the Prompt details that impact the behaviour of the model in one place separate to your application logic.

In [3]:
model = "gpt-3.5-turbo"
temperature = 0
template = [
    {
        "role": "system",
        "content": """Answer the following question factually.

Question: {{question}}

Options:
- {{option_A}}
- {{option_B}}
- {{option_C}}
- {{option_D}}
- {{option_E}}

---

Here is some retrieved information that might be helpful.
Retrieved data:
{{retrieved_data}}

---

Give you answer in 3 sections using the following format. Do not include the quotes or the brackets. Do include the "---" separators.
```
<chosen option verbatim>
---
<clear explanation of why the option is correct and why the other options are incorrect. keep it ELI5.>
---
<quote relevant information snippets from the retrieved data verbatim. every line here should be directly copied from the retrieved data>
```
""",
    }
]

def populate_template(template: list, inputs: dict[str, str]) -> list:
    """Populate a template with input variables."""
    messages = []
    for i, template_message in enumerate(template):
        content = template_message["content"]
        for key, value in inputs.items():
            content = content.replace("{{" + key + "}}", value)
        message = {**template_message, "content": content}
        messages.append(message)
    return messages


## Define the RAG Pipeline

Now we provide the reference RAG pipeline using Chroma and OpenAI that takes a question and returns an answer. This is ultimately what we will evaluate.


In [4]:
def retrieval_tool(question: str) -> str:
    """Retrieve most relevant document from the vector db (Chroma) for the question."""
    response = collection.query(query_texts=[question], n_results=1)
    retrieved_doc = response["documents"][0][0]
    return retrieved_doc


def ask_question(inputs: dict[str, str])-> str:
    """Ask a question and get an answer using a simple RAG pipeline"""
    
    # Retrieve context
    retrieved_data = retrieval_tool(inputs["question"])
    inputs = {**inputs, "retrieved_data": retrieved_data}
    
    # Populate the Prompt template
    messages = populate_template(template, inputs)
    
    # Call OpenAI to get response
    chat_completion = openai.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages,
    )
    answer = chat_completion.choices[0].message.content
    return answer

In [5]:
# Test the pipeline

print(
    ask_question(
        {
            "question": "A 34-year-old male suffers from inherited hemophilia A. He and his wife have three unaffected daughters. What is the probability that the second daughter is a carrier of the disease?",
            'option_A': '0%', 'option_B': '25%', 'option_C': '50%', 'option_D': '75%', 'option_E': '100%'
        }
    )
)

```
50%
---
The probability that the second daughter is a carrier of the disease is 50%. This is because the daughters of a male with hemophilia A will either inherit the disease (if they receive the affected X chromosome) or be carriers (if they receive the normal X chromosome). Since the daughters are unaffected, they must be carriers of the disease.

---
```



# Humanloop Integration

We now integrate Humanloop into the RAG pipeline to first enable logging and then to trigger evaluations against a dataset.


## Initialise the SDK
You will need to set your Humanloop API key in the  `.env` file in the root of the repo. You can retrieve your API key from your [Humanloop organization](https://app.humanloop.com/account/api-keys).


In [9]:
# Init the Humanloop SDK
from humanloop import Humanloop

load_dotenv()
humanloop = Humanloop(api_key=os.getenv("HUMANLOOP_KEY"), base_url="https://neostaging.humanloop.ml/v5")

## Integrate Logging

Below, we add a `humanloop.tools.log(...)` call after the retrieval step to log the retrieved documents to Humanloop
and a `humanloop.prompts.log(...)` call after the chat completion generation.
We also pass in a `session_id` to link these two Logs together.

On running this updated code, Humanloop will now begin to track the versions of your Tool and Prompt and their inputs, outputs and associated metadata. 

In [15]:
# redefine the ask_question function to include logging
import uuid
import inspect

def ask_question(inputs: dict[str, str])-> str:
    """Ask a question and get an answer using a simple RAG pipeline"""
    
    # Retrieve context
    retrieved_data = retrieval_tool(inputs["question"])
    
    # Log the context and retriever details to your Humanloop Tool
    session_id = uuid.uuid4().hex
    humanloop.tools.log(
        path="evals_demo/medqa-retrieval",
        tool={
            "function": {
                "name": "retrieval_tool",
                "description": "Retrieval tool for MedQA.",
            },
            "source_code": inspect.getsource(retrieval_tool),
        },
        output=retrieved_data,
        session_id=session_id,
    )
    
    # Populate the Prompt template
    inputs = {**inputs, "retrieved_data": retrieved_data}
    messages = populate_template(template, inputs)
    
    # Call OpenAI to get a response
    chat_completion = openai.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages,
    )
    answer = chat_completion.choices[0].message.content
    
    # Log the response and Prompt details to your Humanloop Prompt
    humanloop.prompts.log(
        path="evals_demo/medqa-answer",
        prompt={
            "model": model,
            "temperature": temperature,
            "template": template,
        },
        inputs=inputs,
        output=chat_completion.choices[0].message.content,
        output_message=chat_completion.choices[0].message,
        session_id=session_id,
    )
    return answer

In [16]:
# Test the pipeline

print(
    ask_question(
        {
            "question": "A 34-year-old male suffers from inherited hemophilia A. He and his wife have three unaffected daughters. What is the probability that the second daughter is a carrier of the disease?",
            'option_A': '0%', 'option_B': '25%', 'option_C': '50%', 'option_D': '75%', 'option_E': '100%'
        }
    )
)

```
50%
---
Each daughter of a male with hemophilia A has a 50% chance of being a carrier of the disease. In this case, since the daughters are unaffected, the second daughter also has a 50% chance of being a carrier.
---
"InternalMed_Harrison. Table 149-2 lists the currently recommended drugs of choice for prophylaxis of malaria, by destination."
```


## Check your Humanloop workspace

After running this pipeline, you will now see your Prompt and Tool logs in your Humanloop workspace:

If you make changes to your Prompt in code and re-run the pipeline, you will a new version of the Prompt created in Humanloop:


![Prompt Logs](../../assets/images/prompt_version.png)



# Triggering Evaluations

We will now extend our implementation to allow us to run Evaluations on Humanloop against a specific test dataset.

This involves the following steps:
1. Extend our logging to include info needed by Evaluations.
2. Create a Dataset that we can manage and re-use on Humanloop as the source of truth.
3. Create some Evaluators that we can manage and re-use on Humanloop that can provide judgements on the performance of our Pipeline.
4. Trigger an Evaluation and view the results.

Then as you tweak your pipeline in code, this will allow you to easily track and compare the performance of different versions. 

## Extend logging

Add `source_datapoint_id` and `evaluation_id` to the `humanloop.prompt.log(...)` and `humanloop.tool.log(...)` calls.
We do this below by adding the optional `datapoint_id` argument to `ask_question(...)`.

In [22]:
import inspect
import uuid


def ask_question(inputs: dict[str, str], datapoint_id: str | None = None, evaluation_id: str| None = None)-> str:
    """Ask a question and get an answer using a simple RAG pipeline"""
    
    # Retrieve context
    retrieved_data = retrieval_tool(inputs["question"])
    
    # Log the context and retriever details to your Humanloop Tool
    session_id = uuid.uuid4().hex
    humanloop.tools.log(
        path="evals_demo/medqa-retrieval",
        tool={
            "function": {
                "name": "retrieval_tool",
                "description": "Retrieval tool for MedQA.",
            },
            "source_code": inspect.getsource(retrieval_tool),
        },
        output=retrieved_data,
        session_id=session_id,
    )
    
    # Populate the Prompt template
    inputs = {**inputs, "retrieved_data": retrieved_data}
    messages = populate_template(template, inputs)
    
    # Call OpenAI to get a response
    chat_completion = openai.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=messages,
    )
    answer = chat_completion.choices[0].message.content
    
    # Log the response and Prompt details to your Humanloop Prompt
    humanloop.prompts.log(
        path="evals_demo/medqa-answer",
        prompt={
            "model": model,
            "temperature": temperature,
            "template": template,
        },
        inputs=inputs,
        output=chat_completion.choices[0].message.content,
        output_message=chat_completion.choices[0].message,
        session_id=session_id,
        # NB: New arguments to link to Evaluation and Dataset
        source_datapoint_id=datapoint_id,
        evaluation_id=evaluation_id,
    )

    return answer

## Create a dataset
Here we will create a Dataset on Humanloop using the MedQA test dataset. Alternatively you can create a data from Logs on Humanloop, or upload via the UI - see our [guide](https://humanloop.com/docs/v5/evaluation/guides/create-dataset). 

You can then effectively version control your Dataset centrally on Humanloop and hook into it for Evaluation workflows in code and via the UI.

In [18]:
def upload_dataset_to_humanloop():
    df = pd.read_json("../../assets/datapoints.jsonl", lines=True)

    datapoints = [row.to_dict() for _i, row in df.iterrows()][0:20]
    return humanloop.datasets.upsert(
        path="evals_demo/medqa-test",
        datapoints=datapoints,
        commit_message=f"Added {len(datapoints)} datapoints from MedQA test dataset.",
    )


In [19]:
dataset = upload_dataset_to_humanloop()

## Set up Evaluators

Here we will upload some Evaluators defined in code in `assets/evaluators/` so that Humanloop can manage running these for Evaluations (and later for Monitoring!)

Alternatively you can define AI, Code and Human based Evaluators via the UI - see the relevant `How-to guides` on [Evaluations](https://humanloop.com/docs/v5/evaluation/overview) for creating Evaluators of different kinds.

Further you can choose to not host the Evaluator on Humanloop and instead use your own runtime and instead post the results as part of the Evaluation. This can be useful for more complex workflows that require custom dependencies or resources, but lies outside the scope of this tutorial.

In [20]:
def upload_evaluators():
    for evaluator_name, return_type in [
        ("exact_match", "boolean"),
        ("levenshtein", "number"),
    ]:
        with open(f"../../assets/evaluators/{evaluator_name}.py", "r") as f:
            code = f.read()
        humanloop.evaluators.upsert(
            path=f"evals_demo/{evaluator_name}",
            spec={
                "evaluator_type": "python",
                "arguments_type": "target_required",
                "return_type": return_type,
                "code": code,
            },
            commit_message=f"New version from {evaluator_name}.py",
        )

In [21]:
upload_evaluators()

## Run Evaluation

Now we can start to trigger Evaluations on Humanloop using our Dataset and Evaluators:

In [23]:
from tqdm import tqdm


# Create the Evaluation specifying the Dataset and Evaluators to use
evaluation = humanloop.evaluations.create(
    # NB: you can also use the `id` to reference Datasets and Evaluators 
    dataset={"path": "evals_demo/medqa-test"},
    evaluators=[
        {"path": "evals_demo/exact_match"},
        {"path": "evals_demo/levenshtein"},
    ],
)
print(f"Evaluation created: {evaluation.id}")

def populate_evaluation():
    """Run a variation of your Pipeline over the Dataset to populate results"""
    retrieved_dataset = humanloop.datasets.get(
        id=evaluation.dataset.id,
        include_datapoints=True,
    )
    for datapoint in tqdm(retrieved_dataset.datapoints):
        ask_question(
            inputs=datapoint.inputs,
            datapoint_id=datapoint.id,
            evaluation_id=evaluation.id,
        )


Evaluation created: evr_z0uhjU1QQp1IcjolUAhBI


In [24]:
populate_evaluation()

# Then change your pipeline and run this function again to populate additional columns in your Evaluation!

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 50/50 [03:00<00:00,  3.60s/it]


## Get Results and URL
We can not get the aggregate results via the API and the URL to navigate to the Evaluation in the Humanloop UI.

In [26]:
evaluation = humanloop.evaluations.get(id=evaluation.id)
print("URL: ", evaluation.url)

EvaluationResponse(id='evr_z0uhjU1QQp1IcjolUAhBI', dataset=DatasetResponse(path='evals_demo/medqa-test', id='ds_8liIEsObN5SAIq0c6aNm2', directory_id='dir_WFiBDkGinaPccJigBSHQ1', name='medqa-test', version_id='dsv_F2RTWrcSO2e1ZbrcktFXU', type='dataset', environments=[EnvironmentResponse(id='env_NakuPkQXr8w4dYkTAAynO', created_at=datetime.datetime(2024, 4, 29, 8, 1, 19, 415384), name='production', tag='default')], created_at=datetime.datetime(2024, 8, 23, 15, 37, 45, 542145), updated_at=datetime.datetime(2024, 8, 23, 15, 37, 45, 542145), created_by={'id': 'usr_iBV0KNZkHdwXZSIUSaMKG', 'email_address': 'pnhayes+@tcd.ie', 'full_name': 'Peter', 'platform_access': 'user'}, status='committed', last_used_at=datetime.datetime(2024, 8, 23, 15, 37, 45, 542145), commit_message='Added 50 datapoints from MedQA test dataset.', datapoints_count=50, datapoints=None, team_id='tm_b79syTwUvFjr0T1tmT8wq'), evaluatees=[EvaluateeResponse(version={'path': 'evals_demo/medqa-answer', 'id': 'pr_W0F7hDRYvtvRZywP98